In [1]:
import os
from os import path
import pandas as pd

# base_dir = "/home/xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/
# base_dir = "C:/Users/Xin/Dropbox/BrainImaging_ML/multidimensional_age_prediction/"

# input_dir = "result_robustreg_behav_hc2"
# output_dir = "out03_age_prediction"

# input_dir = "result_robustreg_behav_hc2_center_age"
# output_dir = "out03_age_prediction_center_age"

# input_dir = "result_robustreg_behav_hc2_stdz_age"
# output_dir = "out03_age_prediction_stdz_age"

# input_dir = "result_robustreg_behav_hc2_stdz_age_reverse_fa"
# output_dir = "out03_age_prediction_stdz_age_reverse_fa"

input_dir1 = "out01_robustreg_behav_hc3_stdz_age_reverse_fa"
input_dir2 = "out02_robustreg_behav_hc3_stdz_age_reverse_fa_thresh0"
output_dir = "out03_age_prediction_hc3_stdz_age_reverse_fa_thresh0"

# input_dir2 = "out02_robustreg_behav_hc3_stdz_age_reverse_fa"
# output_dir = "out03_age_prediction_hc3_stdz_age_reverse_fa"

# cluster_col_index = 'cluster_gmm_eee3'
# cluster_col_index = 'cluster_gmm_vvi2'
cluster_col_index = 'cluster_gmm_vvv3'


if not os.path.exists(output_dir):
    os.mkdir(output_dir)

#cluster_data = pd.read_csv(input_dir+'/out02_coefs_cluster.csv', index_col = 0)
cluster_data = pd.read_csv(input_dir2+'/out02_coefs_cluster_R_gmm.csv', index_col = 0)

behav_feature = pd.read_csv(input_dir1+'/out01_behav_feature_hc.csv', index_col = 0)


In [2]:
behav_feature.shape

(209, 215)

In [3]:
cluster_data.columns.values[0] = "feature"
cluster_data

,feature,modality,age,age..2.,gender,age.gender,age..2..gender,Rsquare,rsquare_thresh,cluster_gmm_vvv4,cluster_gmm_eee3,cluster_gmm_eve3,cluster_gmm_vvv3,cluster_gmm_vvi2,cluster_gmm_eee2
1,ABF_Accuracy,behav,0.121984,0.029303,0.206584,0.003709,-0.096268,0.052841,True,NaN,NaN,NaN,NaN,NaN,NaN
2,ATT_Accuracy,behav,0.307192,-0.044518,0.198837,-0.090222,-0.076627,0.163884,True,NaN,NaN,NaN,NaN,NaN,NaN
3,WM_Accuracy,behav,0.261293,-0.133767,0.105902,0.040525,-0.074841,0.110568,True,NaN,NaN,NaN,NaN,NaN,NaN
4,VMEM_Accuracy,behav,0.090462,-0.122263,-0.041521,0.002248,-0.009946,-0.000403,False,NaN,NaN,NaN,NaN,NaN,NaN
5,FMEM_Accuracy,behav,0.253389,-0.148188,0.059542,-0.048744,-0.093582,0.087085,True,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Superior.longitudinal.fasciculus.R(tract),tract_fa,-0.313694,0.128628,-0.087989,-0.041649,-0.034377,0.243658,True,2.0,1.0,1.0,1.0,2.0,2.0
209,Uncinate.fasciculus.L(tract),tract_fa,-0.224377,0.197591,0.059521,-0.052759,-0.126983,0.168414,True,3.0,2.0,2.0,2.0,2.0,2.0
210,Uncinate.fasciculus.R(tract),tract_fa,-0.233882,0.185072,-0.012746,-0.088522,-0.062842,0.194901,True,2.0,2.0,2.0,2.0,2.0,2.0
211,Superior.longitudinal.fasciculus.(temporal.pa...,tract_fa,-0.144291,0.107165,-0.003090,0.014678,-0.022324,0.025660,True,3.0,2.0,2.0,2.0,2.0,2.0


In [4]:
from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold

# tune model in n-fold CV:
# n fold CV on hc data:
nfold = 5
seed = 111
kf = KFold(n_splits=nfold, shuffle = True, random_state=seed)
# kf = StratifiedKFold(n_splits=nfold, shuffle = True, random_state=seed)

In [5]:
############################################### SVR: #########################################
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

import numpy as np

# C = np.power(10, np.linspace(-1,1, num = 20))
C = np.power(10, np.linspace(-2,1, num = 20))

epsilon=np.linspace(.001, 1, num = 20)
# epsilon = np.power(10, np.linspace(-5,1, num = 50))

param_grid_svr = dict(C = C, epsilon = epsilon)
# linear model takes very long time to run.
grid_svr = GridSearchCV(estimator=SVR(kernel='linear'), param_grid=param_grid_svr, 
                        n_jobs=3, refit = True, cv = 3)

# use default kernel rbf:
# grid_svr = GridSearchCV(estimator=SVR(), param_grid=param_grid_svr, n_jobs=3, refit = True, cv = 5)



In [6]:
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

def qudratic_r_squared_gender(chro_age, brain_age, gender):
    """
    compute the r squared of curve fit of chro_age vs. brain age.
    1. Use qudratic function to fit brain age with chronological age and gender.
    2. Then, compute r squared of brain age and predicted brain age with qudratic fit model.
    """
    
    def qudratic_fun(x, a, b, c, d, e, f):
        x1, x2 = x
        #return a + b*x1 + c*(x1**2) + d*x2 + e*x1*x2 f*(x1**2)*x2
        return a + b*x1 + c*(x1**2) + d*x2 + e*x1*x2 + f*(x1**2)*x2
    
    popt, pcov = curve_fit(qudratic_fun, (chro_age, gender), brain_age)
    brain_age_pred = qudratic_fun((chro_age, gender), popt[0], popt[1], popt[2], popt[3], popt[4], popt[5])

    r_square = r2_score(brain_age, brain_age_pred)
    return(r_square)
    

In [7]:
# age prediction for each gmm cluster:
cluster_data_gmm = cluster_data.loc[cluster_data[cluster_col_index]<10,:]

cluster_data_gmm[cluster_col_index].unique()

array([1., 2., 3.])

In [8]:
from sklearn import preprocessing

# create empty dataframe to save results:
column_index = pd.MultiIndex.from_product([['Pearson r', 'R square', 'MAE', 'rmse'],
                                           ['cv' + str(i) for i in range(1, kf.n_splits+1, 1)]])

row_index = cluster_data_gmm[cluster_col_index].unique()
result_table = pd.DataFrame(index = row_index, columns = column_index)
result_table = pd.concat([result_table, pd.DataFrame(index = ['all'], columns = column_index)])

print(result_table)

plot_data = pd.DataFrame(columns = ['cluster', 'SUBJID', 'CV', 'chronological age', 'brain age'])

y = behav_feature['age_at_cnb'].values
subjid = behav_feature['SUBJID'].values
# create feature list:
feature_list = []

for icluster in cluster_data_gmm[cluster_col_index].unique():
    
    ifeature = cluster_data_gmm.loc[(cluster_data_gmm[cluster_col_index]==icluster) & 
                                    (cluster_data_gmm['modality']!='behav'), 'feature']
    
    X = pd.concat((behav_feature['Sex'], behav_feature[ifeature]), axis = 1, sort = False).values
    
    feature_list.append([X, icluster])
    
# feature with all clusters:
ifeature = cluster_data_gmm.loc[(cluster_data_gmm[cluster_col_index]<=10) & 
                                    (cluster_data_gmm['modality']!='behav'), 'feature']
    
X = pd.concat((behav_feature['Sex'], behav_feature[ifeature]), axis = 1, sort = False).values  

feature_list.append([X, 'all'])

for i_feature in range(len(feature_list)):
    X = feature_list[i_feature][0]
    icluster = feature_list[i_feature][1]
    
    i=1
    for train_index, test_index in kf.split(X):
        print('cluster:', icluster)
        print('run_model on CV: %d' % i)

        X_train, X_test = X[train_index,:], X[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
        subjid_test = subjid[test_index]

        # normalize X_train and X_test based on mean and sd of X_train. 
        # don't normalize 1st col which is sex, 
        scaler = preprocessing.StandardScaler().fit(X_train[:,1:])
        X_train[:,1:] = scaler.transform(X_train[:,1:])  
        X_test[:,1:] = scaler.transform(X_test[:,1:])

        seed = 7
        np.random.seed(seed)           

        fit_result = grid_svr.fit(X_train, y_train)
        y_prediction = grid_svr.best_estimator_.predict(X_test)

        print("Best: %f using %s" % (fit_result.best_score_, fit_result.best_params_))

        #result_table['R square', 'boot' + str(i)][icluster] = r2_score(y_test, y_prediction)
        #result_table['R square', 'boot' + str(i)][icluster] = qudratic_r_squared(y_test, y_prediction)
        result_table['R square', 'cv' + str(i)][icluster] = qudratic_r_squared_gender(y_test, y_prediction, X_test[:,1])
        result_table['Pearson r', 'cv' + str(i)][icluster] = np.corrcoef(y_test, y_prediction)[0,1]
        result_table['rmse', 'cv' + str(i)][icluster] = np.sqrt(np.mean(np.square(y_test - y_prediction)))
        result_table['MAE', 'cv' + str(i)][icluster] = np.mean(np.abs(y_test - y_prediction))

        plot_data_cv = pd.DataFrame(columns = ['cluster', 'SUBJID', 'CV', 'chronological age', 'brain age'])
        plot_data_cv['chronological age'] = y_test
        plot_data_cv['brain age'] = y_prediction
        plot_data_cv.loc[:,'CV'] = i
        plot_data_cv.loc[:,'cluster'] = icluster
        plot_data_cv['SUBJID'] = subjid_test
        plot_data_cv['Sex'] = X_test[:,0]

        plot_data = pd.concat([plot_data,plot_data_cv])
        i = i+1
        


    Pearson r                     R square                      MAE            \
          cv1  cv2  cv3  cv4  cv5      cv1  cv2  cv3  cv4  cv5  cv1  cv2  cv3   
1.0       NaN  NaN  NaN  NaN  NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
2.0       NaN  NaN  NaN  NaN  NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
3.0       NaN  NaN  NaN  NaN  NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
all       NaN  NaN  NaN  NaN  NaN      NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

              rmse                      
     cv4  cv5  cv1  cv2  cv3  cv4  cv5  
1.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
all  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
cluster: 1.0
run_model on CV: 1
Best: 0.637385 using {'C': 0.01438449888287663, 'epsilon': 0.7896842105263158}
cluster: 1.0
run_model on CV: 2
Best: 0.586569 using {'C': 0.01, 'epsilon': 0.7896842105263158}
cluster: 1.0
run_model on CV: 3
Best: 0.623643 using {'C': 0

In [9]:
X_test[:,0]

array([-1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,
        1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1., -1., -1.,
        1., -1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,
        1.,  1.])

In [10]:
# result_table.to_csv(output_dir+"/out03_result_table_svr_" + cluster_col_index + ".csv")
result_table.to_csv(output_dir+"/out03_result_table_svrlinear_" + cluster_col_index + ".csv")
result_table

Pearson r                                          R square            \
          cv1       cv2       cv3       cv4       cv5       cv1       cv2   
1.0  0.815095  0.866578  0.776577  0.803835  0.765175  0.815497   0.83596   
2.0  0.692183  0.767328   0.63231  0.679844  0.790883  0.644378  0.738219   
3.0  0.371071  0.588344  0.534783  0.216754  0.442959   0.31538  0.418578   
all   0.84023  0.874982  0.820018  0.779784  0.823591  0.807944   0.80873   

                                       MAE                             \
          cv3       cv4       cv5      cv1      cv2      cv3      cv4   
1.0  0.789867  0.856158  0.695903  1.74583   1.4883  1.58907   1.7633   
2.0  0.544336  0.622731  0.698285   2.2294  1.93068  2.02164  2.24238   
3.0  0.446477  0.303765  0.355759  2.79045  2.58111  2.45241  3.26199   
all  0.808306  0.799222  0.786776  1.67879  1.47166  1.50603  1.80837   

                 rmse                                      
         cv5      cv1      cv2      cv3      cv4      cv5  
1.0  1.88038  2.17477  1.84467  2.18884   2.3691  2.25095  
2.0  1.84633  2.71141  2.35954   2.7383   2.9405  2.17704  
3.0  2.84812   3.5046  3.06068  2.89776  3.96957  3.29489  
all  1.54781   2.0501  1.81561  1.99708  2.50431  1.99705

In [11]:
def summary_result(result_table):
    import pandas as pd
    
    column_index = pd.MultiIndex.from_product([['Pearson r', 'R square', 'MAE', 'rmse'], \
                                               ['mean', 'std']])
    
    result_table_summary = pd.DataFrame(columns = column_index)

    result_table_summary['Pearson r', 'mean'] = result_table['Pearson r'].mean(axis = 1)
    result_table_summary['Pearson r', 'std'] = result_table['Pearson r'].std(axis = 1)

    result_table_summary['R square', 'mean'] = result_table['R square'].mean(axis = 1)
    result_table_summary['R square', 'std'] = result_table['R square'].std(axis = 1)
    
    result_table_summary['MAE', 'mean'] = result_table['MAE'].mean(axis = 1)
    result_table_summary['MAE', 'std'] = result_table['MAE'].std(axis = 1)

    result_table_summary['rmse', 'mean'] = result_table['rmse'].mean(axis = 1)
    result_table_summary['rmse', 'std'] = result_table['rmse'].std(axis = 1)
    
    #print('summary_result:')
    #print(result_table_summary)
    
    result_table2 = result_table[['Pearson r', 'R square']].reset_index(level = 0, inplace = False)
    result_table3 = result_table2.rename(columns ={'index':'cluster'}, inplace = False)
    result_accuracy_plot = pd.melt(result_table3, id_vars=['cluster'], var_name = "cv")
    
#     result_table2 = result_table['R square'].reset_index(level = 0, inplace = False)
#     result_table3 = result_table2.rename(columns ={'index':'feature'}, inplace = False)
#     result_r2_plot = pd.melt(result_table3, id_vars=['feature'], \
#                                    value_name = "R square", var_name = "boot")
    #print(result_accuracy_plot)
    return((result_table_summary, result_accuracy_plot))

result_table_summary, result_accuracy_plot = summary_result(result_table)

In [12]:
result_table_summary.to_csv(output_dir+"/out03_result_table_summary_svrlinear_" + cluster_col_index + ".csv")
result_table_summary

Pearson r            R square                 MAE                rmse  \
         mean       std      mean       std      mean       std      mean   
1.0  0.805452  0.039648  0.798677  0.062484  1.693376  0.154553  2.165665   
2.0  0.712510  0.065306  0.649590  0.074252  2.054086  0.177224  2.585358   
3.0  0.430782  0.145914  0.367992  0.062773  2.786816  0.309716  3.345500   
all  0.827721  0.034512  0.802195  0.009483  1.602532  0.139299  2.072830   

               
          std  
1.0  0.195137  
2.0  0.309487  
3.0  0.418012  
all  0.257069

In [13]:
plot_data.to_csv(output_dir+"/out03_scatter_data_svrlinear_" + cluster_col_index + ".csv")
plot_data

,cluster,SUBJID,CV,chronological age,brain age,Sex
0,1,600009963128,1,9.0,11.477941,-1.0
1,1,600484292483,1,18.0,15.379684,1.0
2,1,600561317124,1,19.0,18.615885,1.0
3,1,601064014005,1,12.0,15.189942,-1.0
4,1,601097899414,1,11.0,12.534105,-1.0
...,...,...,...,...,...,...
36,all,608671169548,5,17.0,18.585294,1.0
37,all,608720575588,5,12.0,14.251047,1.0
38,all,609173350200,5,12.0,11.355383,1.0
39,all,609375018060,5,12.0,14.222521,1.0
